In [1]:
# 라이브러리 기본 참조
from hossam import load_data
from pandas import DataFrame, concat, pivot_table, get_dummies
from matplotlib import pyplot as plt
import pandas as pd
from matplotlib import font_manager as fm
import seaborn as sb
import numpy as np
from scipy.stats import t, ttest_ind, normaltest, bartlett, levene
from math import sqrt


# 선형회귀를 위한 참조
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 결과 검정을 위한 참조
from statsmodels.stats.diagnostic import linear_reset
from scipy.stats import zscore, probplot, shapiro, anderson, pearsonr, spearmanr
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from pingouin import anova, welch_anova, pairwise_tukey, pairwise_gameshowell

# VIF값을 계산하기 위한 참조
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statannotations.Annotator import Annotator

# 파이썬 기본 패키지
from itertools import combinations

In [2]:
origin = load_data("diamonds")
print("\n===== 데이터 크기 확인 =====")
print(f"데이터셋 크기: {origin.shape}")
print(f"열 개수: {origin.shape[1]}")
print(f"행 개수: {origin.shape[0]}")
print("\n===== 타입확인 =====")
print(origin.info())
origin.head()

[data] https://data.hossam.kr/data/kaggle/diamonds.xlsx
[desc] 다이아몬드를 커팅, 색상, 투명도, 가격 및 기타 속성 조사한 데이터 셋 (출처: https://www.kaggle.com/datasets/shivam2503/diamonds)

field    description
-------  -------------------------------------------------------------------
price    다이아몬드 가격 (USD, $326 ~ $18,823)
carat    중량 (0.2~5.01)
cut      컷 품질 (Fair, Good, Very Good, Premium, Ideal)
color    색상 등급 - J (worst) to D (best)
clarity  투명도 등급 (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
x        길이 mm (0~10.74)
y        너비 mm (0--58.9)
z        두께 mm (0--31.8)
depth    비율 정보 = z / mean(x, y) = 2 * z / (x + y) (43--79)
table    다이아몬드의 가장 넓은 지점에 비해 상단(테이블)의 너비 (43--95)


===== 데이터 크기 확인 =====
데이터셋 크기: (53940, 10)
열 개수: 10
행 개수: 53940

===== 타입확인 =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   price    53940 non-null  int64  
 1   carat    53940 

,price,carat,cut,color,clarity,x,y,z,depth,table
0,326,0.23,Ideal,E,SI2,3.95,3.98,2.43,61.5,55.0
1,326,0.21,Premium,E,SI1,3.89,3.84,2.31,59.8,61.0
2,327,0.23,Good,E,VS1,4.05,4.07,2.31,56.9,65.0
3,334,0.29,Premium,I,VS2,4.20,4.23,2.63,62.4,58.0
4,335,0.31,Good,J,SI2,4.34,4.35,2.75,63.3,58.0


In [3]:
def ols_report(tbl, data):
    """
    OLS summary(tbl)와 데이터프레임(data)을 받아 회귀계수 표(rdf),
    모델 적합도 요약(result_report), 모형 보고 문장(model_report),
    변수별 보고 리스트(variable_reports)를 반환한다.

    Parameters
    ----------
    tbl : statsmodels.iolib.summary.Summary
        sm.OLS(...).fit().summary() 결과로 생성된 summary 객체.
    data : pandas.DataFrame
        종속변수와 독립변수를 모두 포함한 데이터프레임.

    Returns
    -------
    tuple
        (회귀계수 DataFrame, 적합도 요약 문자열, 모형 보고 문자열, 변수별 보고 리스트)
    """

    # 독립변수 이름(상수항 제외)
    xnames = [n for n in fit.model.exog_names if n != "const"]

    # 독립변수 부분 데이터 (VIF 계산용)
    indi_df = data.filter(xnames)

    # 독립변수 결과를 누적
    variables = []

    for i, v in enumerate(tbl.tables[1].data):
        # 한 행의 변수명 추출 후 목록에 있는지 확인
        name = v[0].strip()
        if name not in xnames:
            continue

        # VIF 계산을 위해 열 인덱스 조회
        j = list(indi_df.columns).index(name)
        vif = variance_inflation_factor(indi_df, j)

        # 유의확률과 별표 표시 함수
        p = float(v[4].strip())

        def stars(p):
            return "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""

        # 한 변수에 대한 보고 정보 추가
        variables.append(
            {
                "종속변수": yname,  # 종속변수 이름 (외부에서 정의)
                "독립변수": name,  # 독립변수 이름
                "B": v[1].strip(),  # 비표준화 회귀계수(B)
                "표준오차": v[2].strip(),  # 계수 표준오차
                "Beta": float(fit.params[name])
                * (
                    data[name].std(ddof=1) / data[yname].std(ddof=1)
                ),  # 표준화 회귀계수(β)
                "t": "%s%s" % (v[3].strip(), stars(p)),  # t-통계량(+별표)
                "p-value": p,  # 계수 유의확률
                "공차": 1 / vif,  # 공차(Tolerance = 1/VIF)
                "vif": vif,  # 분산팽창계수
            }
        )

    rdf = DataFrame(variables)

    # summary 표에서 적합도 정보를 key-value로 추출
    result_dict = {}
    for idx in [0, 2]:
        for item in tbl.tables[idx].data:
            n = len(item)
            for j in range(0, n, 2):
                key = item[j].strip()[:-1]
                value = item[j + 1].strip()
                if not key or not value:
                    continue
                result_dict[key] = value

    # 적합도 보고 문자열 구성
    result_report = (
        f"𝑅({result_dict['R-squared']}), "
        f"𝑅^2({result_dict['Adj. R-squared']}), "
        f"𝐹({result_dict['F-statistic']}), "
        f"유의확률({result_dict['Prob (F-statistic)']}), "
        f"Durbin-Watson({result_dict['Durbin-Watson']})"
    )

    # 모형 보고 문장 구성
    p_f = float(result_dict["Prob (F-statistic)"])
    sig_text = "유의하다" if p_f <= 0.05 else "유의하지 않다"
    p_sign = "≤" if p_f <= 0.05 else ">"

    tpl = (
        "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, "
        "이 회귀모형은 통계적으로 %s(F(%s,%s) = %s, p %s 0.05)."
    )

    model_report = tpl % (
        rdf["종속변수"][0],
        ",".join(list(rdf["독립변수"])),
        sig_text,
        result_dict["Df Model"],
        result_dict["Df Residuals"],
        result_dict["F-statistic"],
        p_sign,
    )

    # 변수별 보고 문장 리스트 구성
    variable_reports = []
    s = "%s의 회귀계수는 %s(p %s 0.05)로, %s에 대하여 %s 예측변인인 것으로 나타났다."

    for i in rdf.index:
        row = rdf.iloc[i]
        p_val = float(row["p-value"])
        p_sign_var = "≤" if p_val < 0.05 else ">"
        sig_word = "유의미한" if p_val < 0.05 else "유의하지 않은"

        variable_reports.append(
            s
            % (
                row["독립변수"],
                row["B"],
                p_sign_var,
                row["종속변수"],
                sig_word,
            )
        )

    return rdf, result_report, model_report, variable_reports

In [4]:
df = origin.filter(["price", "carat", "depth", "table", "x", "y", "z"]).copy()
df.head()

yname = "price"

x = df.drop(yname, axis=1)
y = df[yname]

x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
print(tbl1)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.859
Model:                            OLS   Adj. R-squared:                  0.859
Method:                 Least Squares   F-statistic:                 5.486e+04
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        12:08:42   Log-Likelihood:            -4.7090e+05
No. Observations:               53940   AIC:                         9.418e+05
Df Residuals:                   53933   BIC:                         9.419e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.085e+04    447.562     46.584      0.0

In [5]:
rdf, r, m, v = ols_report(tbl1, df)
display(rdf)
print(m)

,종속변수,독립변수,B,표준오차,Beta,t,p-value,공차,vif
0,price,carat,1.069e+04,63.201,1.269710,169.085***,0.000,0.014259,70.132799
1,price,depth,-203.1541,5.504,-0.072953,-36.910***,0.000,0.001941,515.146274
2,price,table,-102.4457,3.084,-0.057380,-33.216***,0.000,0.001998,500.595341
3,price,x,-1315.6678,43.070,-0.369943,-30.547***,0.000,0.000778,1284.949828
4,price,y,66.3216,25.523,0.018987,2.599**,0.009,0.001883,530.998218
5,price,z,41.6277,44.305,0.007364,0.940,0.347,0.001850,540.677349


price에 대하여 carat,depth,table,x,y,z로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 유의하다(F(6,53933) = 5.486e+04, p ≤ 0.05).


In [6]:
while True:
    rdf, r, m, v = ols_report(tbl1, df)
    if rdf["vif"].max() >= 10:
        df = df.drop(rdf[rdf["vif"] == rdf["vif"].max()]["독립변수"], axis=1)
        x = df.drop(yname, axis=1)
        y = df[yname]
        x_input = sm.add_constant(x)
        fit = sm.OLS(y, x_input).fit()
        tbl1 = fit.summary()
    else:
        break

In [7]:
display(rdf)

,종속변수,독립변수,B,표준오차,Beta,t,p-value,공차,vif
0,price,carat,7820.0379,14.225,0.929149,549.744***,0.0,0.255747,3.910114
1,price,table,-74.3007,3.018,-0.041616,-24.623***,0.0,0.255747,3.910114


In [8]:
df1 = origin.filter(["price", "carat", "cut", "color", "clarity"]).copy()

df1 = get_dummies(
    df1, columns=["cut", "color", "clarity"], dtype="int", drop_first=True
)
df1.head()

,price,carat,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,326,0.23,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
1,326,0.21,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,327,0.23,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,334,0.29,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
4,335,0.31,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0


In [9]:
yname = "price"

x = df1.drop(yname, axis=1)
y = df1[yname]

x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl1 = fit.summary()
print(tbl1)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.916
Method:                 Least Squares   F-statistic:                 3.264e+04
Date:                Thu, 15 Jan 2026   Prob (F-statistic):               0.00
Time:                        12:08:42   Log-Likelihood:            -4.5699e+05
No. Observations:               53940   AIC:                         9.140e+05
Df Residuals:                   53921   BIC:                         9.142e+05
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -7362.8022     51.684   -142.458

In [10]:
rdf, r, m, v = ols_report(tbl1, df1)
display(rdf)
print(m)

,종속변수,독립변수,B,표준오차,Beta,t,p-value,공차,vif
0,price,carat,8886.1289,12.034,1.055819,738.437***,0.0,0.214354,4.665188
1,price,cut_Good,655.7674,33.634,0.047265,19.497***,0.0,0.283897,3.522404
2,price,cut_Ideal,998.2544,30.656,0.122562,32.563***,0.0,0.080458,12.428895
3,price,cut_Premium,869.3959,30.931,0.095068,28.108***,0.0,0.121939,8.200831
4,price,cut_Very Good,848.7169,31.277,0.088696,27.135***,0.0,0.135470,7.381726
5,price,color_E,-211.6825,18.316,-0.020457,-11.557***,0.0,0.426759,2.343241
6,price,color_F,-303.3100,18.509,-0.029011,-16.387***,0.0,0.428726,2.332493
7,price,color_G,-506.1995,18.122,-0.051622,-27.933***,0.0,0.376285,2.657563
8,price,color_H,-978.6977,19.272,-0.088537,-50.784***,0.0,0.449624,2.224078
9,price,color_I,-1440.3019,21.646,-0.108559,-66.538***,0.0,0.537651,1.859942


price에 대하여 carat,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 유의하다(F(18,53921) = 3.264e+04, p ≤ 0.05).


In [11]:
xnames = list(rdf[rdf["p-value"] > 0.05]["독립변수"])
df2 = df1.drop(xnames, axis=1)
x = df2.drop(yname, axis=1)
y = df2[yname]
x_input = sm.add_constant(x)
fit = sm.OLS(y, x_input).fit()
tbl = fit.summary()
rdf, r, m, v = ols_report(tbl, df2)

In [12]:
r

'𝑅(0.916), 𝑅^2(0.916), 𝐹(3.264e+04), 유의확률(0.00), Durbin-Watson(0.907)'